# CMIP5 build intake catalog

Build intake catalog for CMIP5 from manifest files.

In [ ]:
import pandas as pd

## Load manifest

In [ ]:
df = pd.read_csv("../cmip5/manifest_C3S-34a-Lot1_cmip5-monthly-single-level_http_latest.txt.gz", names=["url"])
df

In [ ]:
df.loc[0]["url"]

## Build new dataframe

In [ ]:
df_new = pd.DataFrame(columns = [
    'drs_id', 
    'path',
    'size',
    'project',
    'product',
    'institute',
    'model',
    'experiment',
    'ensemble',
    'time_frequency',
    'variable',
    'version',
    'start_time',
    'end_time',
    'bbox',
    'level',
])
df_new

In [ ]:
df_new["path"] = df["url"].str.replace("http://data.mips.copernicus-climate.eu/thredds/fileServer/esg_c3s-cmip5/", "", regex=False)
df_new.head()

In [ ]:
# c3s-cmip5.output1.MPI-M.MPI-ESM-LR.historical.mon.atmos.Amon.r1i1p1.tas.v20120315

In [ ]:
df_new["drs_id"] = df_new["path"].apply(lambda x: 'c3s-cmip5.' + '.'.join(x.split('/')[0:-1]))
df_new.head()

In [ ]:
df_new["drs_id"][0].split(".")

In [ ]:
df_new['project'] = df_new["drs_id"].apply(lambda x: x.split('.')[0])
df_new['product'] = df_new["drs_id"].apply(lambda x: x.split('.')[1])
df_new['institute'] = df_new["drs_id"].apply(lambda x: x.split('.')[2])
df_new['model'] = df_new["drs_id"].apply(lambda x: x.split('.')[3])
df_new['experiment'] = df_new["drs_id"].apply(lambda x: x.split('.')[4])
df_new['time_frequency'] = df_new["drs_id"].apply(lambda x: x.split('.')[5])
df_new['ensemble'] = df_new["drs_id"].apply(lambda x: x.split('.')[8])
df_new['variable'] = df_new["drs_id"].apply(lambda x: x.split('.')[9])
df_new['version'] = df_new["drs_id"].apply(lambda x: x.split('.')[10])
df_new.head()

In [ ]:
from dateutil.parser import parse

def parse_time(filename):
    time_part = filename.split("_")[-1].split(".nc")[0]
    if "-" not in time_part:
        return None, None
    start, end = time_part.split("-")
    # start year
    start_time = parse(f"{start[0:3]}/{start[4:5]}")
    # end year
    end_time = parse(end)
    return start_time, end_time

In [ ]:
parse("1997/01/01", yearfirst=True)

In [ ]:
df_new["path"][0].split("/")[-1]

In [ ]:
#df_new['start_time'] = df_new["path"].apply(lambda x: parse_time(x.split('/')[-1])[0])
#df_new['end_time'] = df_new["path"].apply(lambda x: parse_time(x.split('/')[-1])[1])
df_new.head()

In [ ]:
cat_name = f"c3s-cmip5_v20220422.csv.gz"
cat_path = f"/tmp/{cat_name}"

df_new.to_csv(cat_path, index=False, compression="gzip")

 ## Load catalog

In [ ]:
df = pd.read_csv(cat_path)
df.head()